In [ ]:
import xgboost as xgb
import shap
import pandas as pd
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor

# --- [1. 데이터 준비] ---
# 분석에 불필요한 컬럼 제거 (ID, 날짜, 타겟, 군집 등)
cols_to_drop = ['발급회원번호', '기준년월', 'Target', 'cluster_label', 'Churn_Score']
# 실제 데이터프레임에 있는 컬럼만 골라서 드롭 (에러 방지)
drop_cols = [c for c in cols_to_drop if c in sampled_df.columns]

X = sampled_df.drop(columns=drop_cols)
X = X.select_dtypes(include=[np.number]) # 숫자형 데이터만 선택
y = sampled_df['Target']

# --- [2. XGBoost 모델 학습] ---
model = xgb.XGBClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=5,
    random_state=42,
    use_label_encoder=False,
    eval_metric='logloss'
)
model.fit(X, y)
print("XGBoost 모델 학습 완료!")

# --- [3. SHAP 중요도(Mean |SHAP|) 계산] ---
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X)

# 각 변수별 평균 절대값(Mean Absolute SHAP) 계산 -> 변수 중요도
shap_importance = np.abs(shap_values).mean(axis=0)

# --- [4. VIF 지수 계산] ---
# 결측치는 평균으로 채워서 계산 (VIF 오류 방지)
X_vif = X.fillna(X.mean())

# 모든 컬럼의 VIF 계산
vif_list = [variance_inflation_factor(X_vif.values, i) for i in range(len(X_vif.columns))]

# --- [5. 데이터프레임 통합 및 정렬] ---
result_df = pd.DataFrame({
    'feature': X.columns,
    'shap_importance': shap_importance,
    'vif': vif_list
})

# SHAP 중요도 기준으로 내림차순 정렬
result_df = result_df.sort_values(by='shap_importance', ascending=False)